In [ ]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.default.utils import keys_by_value, normalize_attrs
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

from profiling_utils import *

In [ ]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
g = Neo4jGraph(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [ ]:
# here we clear the db
res = g.clear()

In [ ]:
nodes = [
    ("a", {"name": "EGFR", "state": "p"}),
    ("b", {"name": "BND"}),
    ("c", {"name": "Grb2", "aa": "S", "loc": 90}),
    ("d", {"name": "SH2"}),
    ("e", {"name": "EGFR"}),
    ("f", {"name": "BND"}),
    ("g", {"name": "Grb2"}),
    ("h", {"name": "WAF1"}),
    ("i", {"name": "BND"}),
    ("j", {"name": "G1-S/CDK", "state": "p"}),
    "k", "l", "m"
    ]
edges = [
    ("a", "b", {"s": "p"}),
    ("d", "b", {"s": "u"}),
    ("d", "c"),
    ("e", "f", {"s": "p"}),
    ("g", "f", {"s": "u"}),
    ("h", "i"),
    ("i", "h"),
    ("j", "h", {"act": {1}}),
    ("j", "i", {"act": {2}}),
    ("k", "l"),
    ("l", "k"),
    ("l", "m"),
    ("m", "l"),
    ("k", "m"),
    ("m", "k"),
    ("e", "b", {"s": "u"})
    ]
g.add_nodes_from(nodes)
g.add_edges_from(edges)

### Profiling clone query

In [ ]:
node = "a"
query = "PROFILE\n" +\
        match_node('x', node) +\
        cloning_query(
            original_var='x',
            clone_var='new_node',
            clone_id=node,
            clone_id_var='uid',
            ignore_naming=True)[0] +\
        return_vars(['uid'])
print(query)
res = g.execute(query).summary()

In [ ]:
print(type(res))

In [ ]:
print(res.metadata["profile"])

In [ ]:
for p in res.profile:
    print(p)

In [ ]:
number_of_DbHits(res.profile)

In [ ]:
print(res.result_available_after + res.result_consumed_after)

### Profiling merge queries (APOC)

In [ ]:
n1 = "a"
n2 = "b"
name = "a_b"
node_list = [n1, n2]

query ="PROFILE\n" +\
        match_nodes({n: n for n in node_list}) + "\n" +\
        merging_query(
            original_vars=node_list,
            merged_var='merged_node',
            merged_id=name,
            merged_id_var='new_id',
            ignore_naming=True)[0] +\
        return_vars(['new_id'])
print(query)
res = g.execute(query).summary()

In [ ]:
number_of_DbHits(res.profile)

In [ ]:
print(res.result_available_after + res.result_consumed_after)

### Profiling merge queries

In [ ]:
n1 = "c"
n2 = "g"
name = "c_g"
node_list = [n1, n2]

query ="PROFILE\n" +\
        match_nodes({n: n for n in node_list}) + "\n" +\
        merging_query1(
            original_vars=node_list,
            merged_var='merged_node',
            merged_id=name,
            merged_id_var='new_id',
            ignore_naming=True)[0] +\
        return_vars(['new_id'])
print(query)
res = g.execute(query).summary()

In [ ]:
number_of_DbHits(res.profile)

In [ ]:
print(res.result_available_after + res.result_consumed_after)

#### Time to match a node with its id

In [ ]:
node = "a"
query = match_node(node, node) + return_vars(node)
print(g.execution_time(query))

#### Time to add a node

In [ ]:
node = "x"
attrs = {"act": {1}}
normalize_attrs(attrs)
query = create_node(node, node, 'new_id', attrs)[0]
print(g.execution_time(query))

#### Time to remove a node

In [ ]:
node = "x"
query = match_node(node, node) + delete_nodes_var(node)
print(g.execution_time(query))

#### Time to add an edge

In [ ]:
source = "a"
target = "c"
attrs = {"act": {1}}
normalize_attrs(attrs)
query = match_nodes({
            source: source,
            target: target
        })
query += create_edge(source, target, attrs)
print(g.execution_time(query))

#### Time to delete an edge

In [ ]:
source = "a"
target = "c"
query = match_edge(source, target, source, target, 'r') 
query += delete_edge_var('r')
print(g.execution_time(query))

#### Time to clone a node

In [ ]:
node = "a"
clone = "a_clone"
query =\
            match_node('x', node) +\
            cloning_query(
                original_var='x',
                clone_var='new_node',
                clone_id=clone,
                clone_id_var='uid')[0] +\
            return_vars(['uid'])
print(g.execution_time(query))

#### Cypher query for cloning (IgnoreNaming=True)

In [ ]:
print("Here we ignore pretty naming of new clones\n")
clone2_res = g.clone_node('a', ignore_naming=True)
print("Created clone by the name: ", clone2_res)
print("Properties of the resulting node: ")
print(g.get_node(clone2_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone2_res, 'b'))

#### Cypher query for cloning

In [ ]:
clone1_res = g.clone_node('a')
print("Created clone by the name: ", clone1_res)
print("Properties of the resulting node: ")
print(g.get_node(clone1_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone1_res, 'b'))